In [14]:
import xarray as xr
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error
from netCDF4 import Dataset
import os
import pandas as pd

In [9]:
def chi_metrics(path_mod):

    # define path and load data
    var_list = ['dir','dif'] # dni: direct normal irradiance DHI: diffuse horizontal irradiance
    varn_obs = ['obs_swddir', 'obs_swddif']
    varn_mod = ['SWDDIR','SWDDIF']

    path_base = "/ss/hsdc/home/tzhang/wrf_solar/"
    path_obs = path_base+"sgpradflux10long_area_mean.c1.20160619_1200UTC.nc"
    path_def = "/S2/gscr2/tzhang/big_data/UQ/tune/runwrf_def/wrfout_d02_2016-06-19_06:00:00"

    fid_obs = Dataset(path_obs)
    fid_def = Dataset(path_def)
    fid_mod = Dataset(path_mod)

    data_obs = {}
    data_def = {}
    data_mod = {}

    for i,varn in enumerate(var_list):
        data_obs[varn] = fid_obs.variables[varn_obs[i]][:]
        data_mod[varn] = np.mean(fid_mod.variables[varn_mod[i]][36:],axis=(1,2))
        data_def[varn] = np.mean(fid_def.variables[varn_mod[i]][36:],axis=(1,2))

    data_obs['tot'] = data_obs['dir'] + data_obs['dif']
    data_mod['tot'] = data_mod['dir'] + data_mod['dif']
    data_def['tot'] = data_def['dir'] + data_def['dif']


    # compute metrics
    metrics_list = ['tot','dir']
    #metrics_list = ['dif','dir']

    
    # old style
    Chi = 0
    for var in metrics_list:
        theta_mod = 0
        for j in range(data_obs[var].shape[0]):
        #theta_mod = mean_absolute_error(data_obs[var], data_mod[var])
            theta_mod += (data_obs[var][j] - data_mod[var][j]) ** 2

        theta_def = 0
        for j in range(data_obs[var].shape[0]):
        #theta_def = mean_absolute_error(data_obs[var], data_def[var])
            theta_def += (data_obs[var][j] - data_def[var][j]) ** 2

        #print(var, theta_mod / theta_def)
        Chi += (theta_mod / theta_def)

    Chi /= len(var_list)
    

    return Chi


In [10]:
aa = calc_metrics("/S2/gscr2/tzhang/big_data/UQ/tune/runwrf_def/wrfout_d02_2016-06-19_06:00:00")

In [15]:
all_cases = os.listdir("/S2/gscr2/tzhang/big_data/UQ/tune/")
data = pd.DataFrame(columns=['vdis','beta_con','chi'])

In [17]:
for case in all_cases:
    fp = open("/S2/gscr2/tzhang/big_data/UQ/tune/"+case+"/namelist.input")
    for i, line in enumerate(fp):
        if i == 85:
            vdis = float(line.split('=')[1])
        if i == 87:
            beta_con = float(line.split('=')[1])
    fp.close() 
        
    chi = chi_metrics("/S2/gscr2/tzhang/big_data/UQ/tune/"+case+"/wrfout_d02_2016-06-19_06:00:00")
    data = data.append({'vdis':vdis,'beta_con':beta_con,'chi':chi},ignore_index=True)